### Topic 7: Tidying Data

#### Merging data frames: join operations

In [1]:
import pandas as pd
from io import StringIO
from IPython.display import display

A_csv = """country,year,cases
Afghanistan,1999,745
Brazil,1999,37737
China,1999,212258
Afghanistan,2000,2666
Brazil,2000,80488
China,2000,213766"""

B_csv = """country,year,population
Afghanistan,1999,19987071
Brazil,1999,172006362
China,1999,1272915272
Afghanistan,2000,20595360
Brazil,2000,174504898
China,2000,1280428583"""

with StringIO(A_csv) as fp:
    A = pd.read_csv(fp)
print("=== A ===")
display(A)

with StringIO(B_csv) as fp:
    B = pd.read_csv(fp)
print("\n=== B ===")
display(B)

# pandas merging two dataframes
C = A.merge(B, on=['country', 'year'])
print("\n=== C = merge(A, B) ===")
display(C)

=== A ===


,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766



=== B ===


,country,year,population
0,Afghanistan,1999,19987071
1,Brazil,1999,172006362
2,China,1999,1272915272
3,Afghanistan,2000,20595360
4,Brazil,2000,174504898
5,China,2000,1280428583



=== C = merge(A, B) ===


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


#### Apply function
- Used to apply a function to a dataframe or series

In [2]:
# convert the year column in C into an abbrievated two-digit form
G = C.copy() # If you do not use copy function the original data frame is modified
G['year'] = G['year'].apply(lambda x: "'{:02d}".format(x % 100))
display(G)

,country,year,cases,population
0,Afghanistan,'99,745,19987071
1,Brazil,'99,37737,172006362
2,China,'99,212258,1272915272
3,Afghanistan,'00,2666,20595360
4,Brazil,'00,80488,174504898
5,China,'00,213766,1280428583


In [3]:
H = G.copy()
H['prevalence'] = H.apply(lambda x: x['cases'] / x['population'], axis = 1)
display(H)

,country,year,cases,population,prevalence
0,Afghanistan,'99,745,19987071,0.000037
1,Brazil,'99,37737,172006362,0.000219
2,China,'99,212258,1272915272,0.000167
3,Afghanistan,'00,2666,20595360,0.000129
4,Brazil,'00,80488,174504898,0.000461
5,China,'00,213766,1280428583,0.000167


#### Canonicalize a tibble

In [4]:
def canonicalize_tibble(X):
    # Sort variables by name, ascending from left to right
    var_names = sorted(X.columns)
    Y = X[var_names].copy()

    # Sort rows by variables from top to bottom
    Y.sort_values(by=var_names, inplace=True)
    Y.set_index([list(range(0, len(Y)))], inplace = True)
    return Y

# Test: `canonicalize_tibble_test`
data = {'': [2, 0, 3, 1],
        'c': ['hat', 'rat', 'cat', 'bat'],
        'a': ['x', 'y', 'x', 'x'],
        'b': [1, 4, 2, 2]}
X = pd.DataFrame(data, columns = ['c','a','b'])
print(canonicalize_tibble(X))

   a  b    c
0  x  1  hat
1  x  2  bat
2  x  2  cat
3  y  4  rat


#### Determine if two tibbles are equivalent
- First canonicalize the two tibbles before checking

In [5]:
def tibbles_are_equivalent(A, B):
    """Given two tidy tables ('tibbles'), returns True iff they are
    equivalent.
    """
    A_copy = canonicalize_tibble(A)
    B_copy = canonicalize_tibble(B)
    return A_copy.equals(B_copy)

#### Melting - turn columns into rows

In [6]:
def melt(df, col_vals, key, value):
    assert type(df) is pd.DataFrame
    keep_vars = df.columns.difference(col_vals)
    melted_sections = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted_sections.append(melted_c)
    melted = pd.concat(melted_sections)
    return melted

# table to melt
table_data = {'': [0, 1, 2],
              'country': ['Afghanistan', 'Brazil', 'China'], 
              '1999': [745, 37737, 212258],
              '2000': [2666, 80488, 213766]
             }
table_data = pd.DataFrame(table_data, columns = ['country', '1999', '2000'])                                        
display(table_data)

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


In [7]:
# test melt
melted_table = melt(table_data, col_vals = ['1999', '2000'], key = 'year', value = 'cases')
display(melted_table)

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
0,Afghanistan,2000,2666
1,Brazil,2000,80488
2,China,2000,213766


#### Casting - turn a long tibble into wide format 

In [8]:
def cast(df, key, value, join_how='outer'):
    """Casts the input data frame into a tibble,
    given the key column and value column.
    """
    assert type(df) is pd.DataFrame
    assert key in df.columns and value in df.columns
    assert join_how in ['outer', 'inner']
    
    fixed_vars = df.columns.difference([key, value])
    tibble = pd.DataFrame(columns=fixed_vars) # empty frame
    
    new_vars = df[key].unique()
    for v in new_vars:
        df_v = df[df[key] == v]
        del df_v[key]
        df_v = df_v.rename(columns = {value: v})
        tibble = tibble.merge(df_v, on = list(fixed_vars), how=join_how)    
    return tibble

```python

# Data Manipulation with Pandas
pd.Series(data, index=index) # constructing series object

# initial exploration and data clean up
data = pd.read_csv('csv_file_name') # load csv file
data.head() # check data head         
data.info() # check info
data.sample(5) # get data sample

# get a list of the unique candidates in column named 'cand_name'
unique_candidate = data['cand_name'].unique()

# get list of top occupation
data['contrb_occupation'].value_counts()

# determine top 7 largest donors
largest_donors = data['contb_receipt_amt'].nlargest(7)
```

```python

# pandas functions
cafes6 = cafes4.reset_index(drop=True) # reset the index from the string names to integers
pd.concat([cafes_cheap, cafes_pricey], join='outer') # concatenate two df along rows, default to 'outer' join
pd.merge(left_df, right_df, how='inner', on=None, left_on=None, right_on=None) # merge using SQL flavor type join 

# handling missing values
data.isnull() # check for missing values
data.dropna() # dropping missing values, axis = 0 for rows, axis = 1 for columns
data.fillna(0) # fill missing values with 0
```

#### Series as a specialized dictionary

In [9]:
# constructing new series
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
display(population)
print()

area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
display(area)

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

In [10]:
# create dataframe from series
states = pd.DataFrame({'population': population,
                       'area': area})
states

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


#### Data Selection in Series

In [11]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [12]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [13]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [14]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [15]:
# fancy indexing
data[['a', 'd']]

a    0.25
d    1.00
dtype: float64

### Topic 9: Relational Data

#### What makes up an SQL query?

**SQL query clauses -- Order of appearance in the query**

There are 6 possible clauses in an SQL query.

They must appear in the query in the following order:

1. ***SELECT*** -- Which columns/data elements are to be included in the result set.


2. ***FROM*** -- The tables which are the source of the data to be returned


3. ***WHERE***
    
    a. The columns from different tables that are equivalent and define how the tables are joined together.
    
    b. Any filtering criteria for the query, to return a subset of the data. Note that this filtering is done PRIOR to any aggregations.


4. ***GROUP BY*** -- If aggregating, these are the columns that the aggregations are based on.


5. ***HAVING*** -- Filtering on data after aggregations have been performed.


6. ***ORDER BY*** -- Sorting the data.


There is a 7th clause, **which is not universal to all databases,** but it is available in SQLite. It is the **LIMIT** clause, which tells the database how many rows to return. It would be last in the order of the query, and would also execute last.

#### Types of Joins
- INNER join: return only rows that are in both tables
- OUTER join (or Full Outer join or Full join): returns all the rows from both tables, whether or not there is a match. NULL values returned for non-matching columns
- LEFT join: return all of the rows from the left table, and records that match from the right table. Non-matching records will return a NULL values
- RIGHT join: return all of the rows from the right table, and records that match from the left table. Non-matching records will return a NULL values
- CROSS join: matches every row of the left table with every row of the right table and returns their Cartesian product. It's also known as Cartesian JOIN.
- SELF join: allows you to join a table to itself as if the tables were two different tables.

```python
import sqlite3 as db

# create connection to database
conn = db.connect('name_of_database.db')

# Create a 'cursor' for executing commands
c = conn.cursor()

# creating tables "Students"
c.execute('DROP TABLE IF EXISTS Students')
c.execute("CREATE TABLE Students (gtid INTEGER, name TEXT)")

# populate table
c.execute("INSERT INTO Students VALUES (123, 'Vuduc')")
c.execute("INSERT INTO Students VALUES (456, 'Chau')")

# inserting many data into table
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?)', more_students)

# save insertion into database
conn.commit()
```

#### Select statement

```python
# connect to database
import sqlite3 as db
disk_engine = db.connect('file:{}?mode=ro'.format(DB_FILENAME), uri=True)

# query SQL with SQL reader built into pandas
df = pd.read_sql_query('SELECT * FROM data', disk_engine)

# find UNIQUE "city" values from "data" table
query = 'SELECT DISTINCT City FROM data'
df = pd.read_sql_query(query, disk_engine)

# character-case conversion and GROUP BY
query = '''
  SELECT LOWER(ComplaintType), LOWER(Descriptor), LOWER(Agency)
    FROM data
    GROUP BY LOWER(ComplaintType)
    LIMIT 10
'''
df = pd.read_sql_query(query, disk_engine)

# HAVING clause
query2 = '''
    SELECT ComplaintType, COUNT(*)
      FROM (SELECT DISTINCT ComplaintType FROM data)
      GROUP BY LOWER(ComplaintType)
      HAVING COUNT(*) >= 2
'''
df2 = pd.read_sql_query(query2, disk_engine)

# IN operator
query = '''
    SELECT DISTINCT ComplaintType
      FROM data
      WHERE LOWER(ComplaintType) IN ("plumbing", "elevator")
'''
df = pd.read_sql_query(query, disk_engine)

# ORDER BY clause
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints DESC
'''
df = pd.read_sql_query(query, disk_engine)
df.tail()

# the LIKE operator
query = '''
  SELECT LOWER(ComplaintType) AS type, COUNT(*) AS freq
    FROM data
    WHERE LOWER(ComplaintType) LIKE '%noise%'
    GROUP BY type
    ORDER BY -freq
'''
df_noisy = pd.read_sql_query(query, disk_engine)

# DATETIME in SQL
query = '''
        SELECT STRFTIME('%H', CreatedDate) AS hour, COUNT(ComplaintType) AS count
        FROM data
        WHERE LOWER(ComplaintType) LIKE '%noise%'
        AND STRFTIME('%H:%M:%f', CreatedDate) <> '00:00:00.000'
        GROUP BY hour
'''

df_noisy_by_hour = pd.read_sql_query (query, disk_engine)

#Write an SQL query find the ten (10) teams that have the highest average away-scores since the year 2000. Your query should satisfy the following criteria:
#It should return two columns: team: The name of the team, ave_goals: The team's average number of goals in "away" games. An "away game" is one in which the team's name appars in away_team and the game takes place at a "non-neutral site" (neutral value equals FALSE).
#It should only include teams that have played at least 30 away matches.
#It should round the average goals value (ave_goals) to three decimal places.
#It should only return the top 10 teams in descending order by average away-goals.
#It should only consider games played since 2000 (including the year 2000).
query_top10_away = """
            SELECT away_team AS team, ROUND(AVG(away_score), 3) AS ave_goals
            FROM soccer_results
            WHERE STRFTIME('%Y', date) >= '2000' AND neutral = 'FALSE'
            GROUP BY away_team
            HAVING COUNT(*) >= 30
            ORDER BY ave_goals DESC
            LIMIT 10
        """

# Suppose we are now interested in the top 10 teams having the best goal differential, between the years 2012 and 2018 (both inclusive). A team's goal differential is the difference between the total number of goals it scored and the total number it conceded across all games (in the requested years).
# Complete the function, best_goal_differential(), below, so that it returns a pandas dataframe containing the top 10 teams by goal differential, sorted in descending order of differential. The dataframe should have two columns: team, which holds the team's name, and differential, which holds its overall goal differential.
def best_goal_differential():
    ###
    ### YOUR CODE HERE
    ###
    sql_query = """WITH home as (SELECT home_team, SUM((home_score-away_score)) as diff
                         FROM soccer_results 
                         WHERE strftime('%Y', date) >= '2012' AND strftime('%Y', date) <= '2018' 
                         GROUP BY home_team),
                away as (SELECT away_team, SUM((away_score-home_score)) as diff
                        FROM soccer_results 
                        WHERE strftime('%Y', date) >= '2012' AND strftime('%Y', date) <= '2018' 
                        GROUP BY away_team)
                SELECT away_team as team, (away.diff + home.diff) as differential
                FROM away, home
                WHERE away.away_team = home.home_team 
                GROUP BY away_team
                ORDER BY differential DESC
                LIMIT 10"""
    df = pd.read_sql_query(sql_query, disk_engine)
    return df
```





#### Join Queries

```python
query = '''
        SELECT Students.name, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid
        AND Takes.course = 'CSE 6040'
        '''

or 

query = '''
        SELECT Students.name, Takes.grade
        FROM Students
        JOIN Takes
        ON Students.gtid = Takes.gtid
        '''


c.execute(query)
results = c.fetchall()
results
```

### Topic 10: Numerical Computing with Numpy/Scipy 

#### Creating multidimensional arrays

In [16]:
import numpy as np
display(np.__version__)

# Create a two-dimensional array of size 3 rows x 4 columns:
B = np.array([[0, 1, 2, 3],
              [4, 5, 6, 7],
              [8, 9, 10, 11]])

print(B)
print()
print('Dimension of B:', B.ndim)
print('Size of B:', B.size)
print('Shape of B:', B.shape)
print('Length of B:', len(B))

'1.21.5'

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

Dimension of B: 2
Size of B: 12
Shape of B: (3, 4)
Length of B: 3


#### Index slicing

Consider the following $6 \times 6$ matrix, which has 4 different subsets highlighted.

<img src="https://github.com/cse6040/labs-fa17/raw/master/lab10-numpy/slicing-exercise.png" alt="Exercise: Extract these slices" width="240">

For each subset illustrated above, write an indexing or slicing expression that extracts the subset. Store the result of each slice into `Z_green`, `Z_red`, `Z_orange`, and `Z_cyan`.

In [17]:
Z= np.array([[0,1,2,3,4,5],
             [10,11,12,13,14,15],
             [20,21,22,23,24,25],
             [30,31,32,33,34,35],
             [40,41,42,43,44,45],
             [50,51,52,53,54,55]])

# Construct `Z_green`, `Z_red`, `Z_orange`, and `Z_cyan`:
###
### YOUR CODE HERE
###
Z_green = Z[2::2, 0::2]
Z_red = Z[:, 2]
Z_orange = Z[0, 3:5]
Z_cyan = Z[4::, 4::]
print(Z_red)

[ 2 12 22 32 42 52]


Complete the prime number sieve algorithm, which is illustrated below.

<img src="https://github.com/cse6040/labs-fa17/raw/master/lab10-numpy/prime-sieve.png" alt="Exercise: Extract these slices" width="480">

That is, given a positive integer $n$, the algorithm iterates from $i \in \{2, 3, 4, \ldots, \left\lfloor\sqrt{n}\right\rfloor\}$, repeatedly "crossing out" values that are strict multiples of $i$. "Crossing out" means maintaining an array of, say, booleans, and setting values that are multiples of $i$ to `False`.

In [18]:
import numpy as np
from math import sqrt, floor

def sieve(n):
    """
    Returns the prime number 'sieve' shown above.
    
    That is, this function returns an array `X[0:n+1]`
    such that `X[i]` is true if and only if `i` is prime.
    """
    is_prime = np.empty(n+1, dtype=bool) # the "sieve"

    # Initial values
    is_prime[0:2] = False # {0, 1} are _not_ considered prime
    is_prime[2:] = True # All other values might be prime

    # Implement the sieving loop
    ###
    ### YOUR CODE HERE
    ###
        
    # Implement the sieving loop
    for i in range(2, int(sqrt(n))):
        is_prime[2*i::i] = False
    
    return is_prime

# Prints your primes
print("==> Primes through 20:\n", np.nonzero(sieve(20))[0])

==> Primes through 20:
 [ 2  3  5  7 11 13 17 19]


#### Aggregation and functions in Numpy

In [19]:
from numpy.random import default_rng
rng = default_rng()

A = rng.integers(-10, 10, size=(4, 3)) # return random integers from -10 (inclusive) to 10 (exclusive)
print(A)
print("np.max =", np.max(A, axis=1),"; np.amax =", np.amax(A)) # np.max() and np.amax() are synonyms
print("np.min =",np.min(A),"; np.amin =", np.amin(A)) # same
print("np.sum =",np.sum(A))
print("np.mean =",np.mean(A))
print("np.std =",np.std(A))

[[ 5 -3 -8]
 [ 3  2 -1]
 [-8 -1  0]
 [ 8  7 -3]]
np.max = [5 3 0 8] ; np.amax = 8
np.min = -8 ; np.amin = -8
np.sum = 1
np.mean = 0.08333333333333333
np.std = 4.990964057396349


In [20]:
print("Max in each column:", np.amax(A, axis=0)) # i.e., aggregate along axis 0, the rows, producing column maxes
print("Max in each row:", np.amax(A, axis=1)) # i.e., aggregate along axis 1, the columns, producing row maxes

Max in each column: [8 7 0]
Max in each row: [5 3 0 8]


#### Elementwise product

The elementwise product is an array `C` such that `C[i, j] = A[i, j] * B[i, j]`, which can be invoked via the `*` operator:

In [21]:
B = rng.integers(-10, 10, size=A.shape)
print(B)

A * B

[[ -9   2   3]
 [ -7 -10   1]
 [  4   3   8]
 [  7  -4   2]]


array([[-45,  -6, -24],
       [-21, -20,  -1],
       [-32,  -3,   0],
       [ 56, -28,  -6]], dtype=int64)

#### Matrix multiplication

In [22]:
B = rng.integers(-10, 10, size=(A.shape[1], 5))
print(f"B ==\n{B}")

C = A.dot(B)
print(C)

B ==
[[ 3 -3  2  9 -6]
 [ 3 -3 -9  8  9]
 [ 4 -1  5 -4  9]]
[[ -26    2   -3   53 -129]
 [  11  -14  -17   47   -9]
 [ -27   27   -7  -80   39]
 [  33  -42  -62  140  -12]]


#### Dense matrix storage: Column-major versus row-major layouts

In [23]:
# Storing matrix in a 1-D array

# scale the col index by the number of rows and 
# add row index for the col-major order
def linearize_colmajor(i, j, m, n): # calculate `u`
    """
    Returns the linear index for the `(i, j)` entry of
    an `m`-by-`n` matrix stored in column-major order.
    """
    return i + (j*m)

# scale the row index by the number of columns and 
# add column index for the row-major order.
def linearize_rowmajor(i, j, m, n): # calculate `v`
    """
    Returns the linear index for the `(i, j)` entry of
    an `m`-by-`n` matrix stored in row-major order.
    """
    return i*n + j

In [24]:
# scales each column in matrix A,  A(:, j) by  j
def scale_colwise(A):
    """Given a Numpy matrix `A`, visits each column `A[:, j]`
    and scales it by `j`."""
    assert type(A) is np.ndarray
    
    n_cols = A.shape[1] # number of columns
    ###
    ### YOUR CODE HERE
    ###
    for j in range(n_cols):
        A[:, j] *= j
    return A

In [25]:
# Generate random values, for use in populating the matrix and vector
n = 2
from random import gauss

# Native Python, using lists
A_py = [gauss(0, 1) for i in range(n*n)] # Assume: Column-major
x_py = [gauss(0, 1) for i in range(n)]
print(A_py)
print(x_py)

[2.0871421375064965, -0.25541972743089897, 0.1416295567009787, 0.15729245300412714]
[-0.5998696164659403, -1.3531482620992445]


In [26]:
# Convert values into Numpy arrays in column-major order
A_np = np.reshape(A_py, (n, n), order='C') # row-major order
B_np = np.reshape(A_py, (n, n), order='F') # column-major order
x_np = np.reshape(x_py, (n, 1), order='F')
print(A_np)
print()
print(B_np)
print()
print(x_np)

[[ 2.08714214 -0.25541973]
 [ 0.14162956  0.15729245]]

[[ 2.08714214  0.14162956]
 [-0.25541973  0.15729245]]

[[-0.59986962]
 [-1.35314826]]


In [27]:
# Matrix-vector multiply
mat_vec_multiply = A_np.dot(x_np)

def matvec_py(m, n, A, x):
    """
    Native Python-based matrix-vector multiply, using lists.
    The dimensions of the matrix A are m-by-n, and x is a
    vector of length n.
    """
    assert type(A) is list and all([type(aij) is float for aij in A])
    assert type(x) is list
    assert len(x) >= n
    assert len(A) >= (m*n)

    y = [0.] * m
    A_np = np.reshape(A, (m, n), order = 'F')
    x_np = np.reshape(x, (n, 1), order = 'F')
    y = A_np.dot(x_np)
    return y